# Covid 19 - Dataset Processing

by: Leandro Arruda
Special thanks for Mario Filho https://github.com/ledmaster

## Exploring Distance between Countries and  Confirmed cases

[https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density]

[https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset]

[https://www.kaggle.com/fernandol/countries-of-the-world]

[https://www.kaggle.com/max-mind/world-cities-database]

[https://www.kaggle.com/andradaolteanu/iso-country-codes-global]

[https://www.kaggle.com/folaraz/world-countries-and-continents-details]

I explored several datasets, looking for certain characteristics that might influences CoVid-19 prediction.
In result, I ended up with the idea to look at:
    * The proximity of countries
    * Countries which had SARS. (Since there're some common characteristics. I.e. Both are a Coronavirus infection, the initial cases started in the same countries, among others)
    * Population density. (Observing the spread characteristic)

## Importing Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
from geopy.distance import geodesic
import os
#plt.figure(figsize=(16,6))

## Reading the datasets

In [2]:
# Input data files are available in the "../data/" directory.

for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.\data\2019_nCoV_data.csv
.\data\countries-lat-lon.csv
.\data\countries.csv
.\data\countries_and_continents.csv
.\data\covid_19_data.csv
.\data\Population.csv
.\data\sars.csv
.\data\time_series_covid_19_confirmed.csv
.\data\time_series_covid_19_deaths.csv
.\data\time_series_covid_19_recovered.csv


In [3]:
covid = pd.read_csv('./data/covid_19_data.csv')
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [4]:
countries = pd.read_csv("./data/countries.csv")
countries_loc = pd.read_csv("./data/countries-lat-lon.csv", index_col=0)
countries.head()

,name,official_name_en,official_name_fr,ISO3166-1-Alpha-2,ISO3166-1-Alpha-3,M49,ITU,MARC,WMO,DS,...,ISO4217-currency_minor_unit,ISO4217-currency_name,ISO4217-currency_numeric_code,is_independent,Capital,Continent,TLD,Languages,Geoname ID,EDGAR
0,NaN,Channel Islands,Îles Anglo-Normandes,NaN,NaN,830,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Sark,Sercq,NaN,NaN,680,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Afghanistan,Afghanistan,AF,AFG,4,AFG,af,AF,AFG,...,2.0,Afghani,971.0,Yes,Kabul,AS,.af,"fa-AF,ps,uz-AF,tk",1149361.0,B2
3,Albania,Albania,Albanie,AL,ALB,8,ALB,aa,AB,AL,...,2.0,Lek,8.0,Yes,Tirana,EU,.al,"sq,el",783754.0,B3
4,Algeria,Algeria,Algérie,DZ,DZA,12,ALG,ae,AL,DZ,...,2.0,Algerian Dinar,12.0,Yes,Algiers,AF,.dz,ar-DZ,2589581.0,B4


## Data Cleaning

### Preparing COVID dataset merging other datasets

In [5]:
covid['ObservationDate'] = pd.to_datetime(covid['ObservationDate'], format='%m/%d/%Y')
covid.loc[covid['Country/Region'] == 'China', 'Country/Region'] = 'Mainland China'
covid.loc[covid['Country/Region'] == 'Côte d’Ivoire', 'Country/Region'] = 'Ivory Coast'

In [6]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [8]:
covid = covid[covid['Confirmed'] > 0]

In [9]:
covid['Country/Region'].value_counts().head(10)

Mainland China    1067
US                 298
Australia          119
Canada              82
Macau               35
South Korea         35
Japan               35
Taiwan              35
Thailand            35
Hong Kong           34
Name: Country/Region, dtype: int64

In [10]:
#covid = covid.drop_duplicates("Country/Region")
covid = covid[covid["Country/Region"] != "Others"]
covid = covid.reset_index(drop=True)
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,6,2020-01-22,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0


### Preparing World Countries & Location to Merge into COVID dataset

**Renaming Name column**

In [11]:
countries_loc = countries_loc.rename(columns={"name": "Country/Region"})
countries_loc = countries_loc.dropna()
countries_loc.head()

,longitude,latitude,Country/Region
2,33.939110,67.709953,Afghanistan
3,41.153332,20.168331,Albania
4,28.033886,1.659626,Algeria
5,-14.270972,-170.132217,American Samoa
6,42.506285,1.521801,Andorra


In [12]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,6,2020-01-22,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0


In [13]:
countries_loc.loc[countries_loc['Country/Region'] == 'China', 'Country/Region'] = 'Mainland China'
countries_loc.loc[countries_loc['Country/Region'] == 'Côte d’Ivoire', 'Country/Region'] = 'Ivory Coast'

### Preparing SARS Dataset to Merge into COVID dataset

In [14]:
sars = pd.read_csv('./data/sars.csv')
sars['Country or region'] = sars['Country or region'].str.strip()
sars.loc[sars['Country or region'] == 'China', 'Country or region'] = 'Mainland China'
sars.loc[sars['Country or region'] == 'United States', 'Country or region'] = 'US'
sars.loc[sars['Country or region'] == 'United Kingdom', 'Country or region'] = 'UK'
sars['SARS'] = 1
sars = sars.rename(columns={'Country or region': "Country/Region"})
sars = sars.rename(columns={'Cases': "SARS_Cases"})
sars = sars.rename(columns={'Deaths': "SARS_Deaths"})
sars = sars.rename(columns={'Fatality (%)': "SARS_Fatality"})
sars.head()

,Country/Region,SARS_Cases,SARS_Deaths,SARS_Fatality,SARS
0,Mainland China,5327,349,6.6,1
1,Hong Kong,1755,299,17.0,1
2,Taiwan,346,73,21.1,1
3,Canada,251,43,17.1,1
4,Singapore,238,33,13.9,1


In [15]:
countries_loc = countries_loc.merge(sars, how='left', on='Country/Region')


In [16]:
countries_loc.loc[countries_loc['Country/Region']=='Mainland China']

,longitude,latitude,Country/Region,SARS_Cases,SARS_Deaths,SARS_Fatality,SARS
45,35.86166,104.195397,Mainland China,5327.0,349.0,6.6,1.0


In [17]:
covid = covid.merge(countries_loc, how='left', on='Country/Region')

In [18]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,longitude,latitude,SARS_Cases,SARS_Deaths,SARS_Fatality,SARS
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.86166,104.195397,5327.0,349.0,6.6,1.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,35.86166,104.195397,5327.0,349.0,6.6,1.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,35.86166,104.195397,5327.0,349.0,6.6,1.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.86166,104.195397,5327.0,349.0,6.6,1.0
4,6,2020-01-22,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0,35.86166,104.195397,5327.0,349.0,6.6,1.0


### Preparing Population Dataset to Merge into COVID dataset

In [19]:
population = pd.read_csv('./data/population.csv')
population.head()

,Country,Population,Land Area,Population Density,% of land arable,Arable Land,Real Population Density 2016
0,Singapore,5612253,682.7,7916.0,0.79%,560,0.000
1,Hong Kong S.A.R.,7391700,1042.0,7040.0,2.86%,3000,0.000
2,Bahrain,1492584,665.0,1936.0,2.06%,1600,0.001
3,Seychelles,95843,455.0,208.0,0.33%,150,0.002
4,Kuwait,4136528,17820.0,232.0,0.45%,8000,0.002


In [20]:
population.loc[population['Country'] == 'China', 'Country'] = 'Mainland China'
population.loc[population['Country'] == 'United States', 'Country'] = 'US'
population.loc[population['Country'] == 'United Kingdom', 'Country'] = 'UK'

In [21]:
population = population.rename(columns={"Country": "Country/Region"})

In [22]:
population = population.drop(columns=['Land Area', '% of land arable','Arable Land','Real Population Density 2016'])

In [23]:
population.head()

,Country/Region,Population,Population Density
0,Singapore,5612253,7916.0
1,Hong Kong S.A.R.,7391700,7040.0
2,Bahrain,1492584,1936.0
3,Seychelles,95843,208.0
4,Kuwait,4136528,232.0


In [24]:
covid = covid.merge(population, how='left', on='Country/Region')

In [25]:
covid

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,longitude,latitude,SARS_Cases,SARS_Deaths,SARS_Fatality,SARS,Population,Population Density
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
4,6,2020-01-22,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
5,7,2020-01-22,Guangxi,Mainland China,1/22/2020 17:00,2.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
6,8,2020-01-22,Guizhou,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
7,9,2020-01-22,Hainan,Mainland China,1/22/2020 17:00,4.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
8,10,2020-01-22,Hebei,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0
9,12,2020-01-22,Henan,Mainland China,1/22/2020 17:00,5.0,0.0,0.0,35.861660,104.195397,5327.0,349.0,6.6,1.0,1.386395e+09,148.0


### Saving the new CoViD-19 dataset

In [28]:
covid.to_csv('./data/covid_19_processed.csv')